In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import random
import matplotlib.dates as mdates
from tensorflow.keras.layers import Attention
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LSTM, Bidirectional, Attention
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, LSTM, Dropout, Dense, Attention, Bidirectional, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
data = pd.read_csv(r'Database2.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.sort_values('Date', inplace=True)

data['ExchangeRate_Short_MA'] = data['ExchangeRate'].rolling(window=20, min_periods=1).mean()
data['ExchangeRate_Long_MA'] = data['ExchangeRate'].rolling(window=80, min_periods=1).mean()

data.dropna(inplace=True)

In [ ]:
selected_features = [
    'Price', 'STI', 'GoldPrice', 'DXI',
    'USD_EUR_ExchangeRate', 'USD_CNY_ExchangeRate',
    'ExchangeRate_Short_MA', 'ExchangeRate_Long_MA',
    'impact_score'
]
features = data[selected_features]
target = data['ExchangeRate']

features.dropna(inplace=True)
target.dropna(inplace=True)

In [ ]:
scaler_features = MinMaxScaler()
features_scaled = scaler_features.fit_transform(features)
scaler_target = MinMaxScaler()
target_scaled = scaler_target.fit_transform(target.values.reshape(-1, 1))

In [ ]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(features_scaled, target_scaled.ravel())
rf_predictions = rf_model.predict(features_scaled)
rf_predictions_reshaped = rf_predictions.reshape(-1, 1)

In [ ]:
extended_features = np.hstack((features_scaled, rf_predictions_reshaped))
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X[i:(i + time_steps), :]
        Xs.append(v)
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

In [ ]:
def plot_predictions(real_vals, lstm_predicted_vals, title="Predictions vs. Actual", num_dates=10):
    plt.figure(figsize=(14, 7))


    dates = data['Date'].iloc[train_size:train_size + len(real_vals)].map(mdates.date2num)

    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))


    baseline_predicted_vals = np.roll(real_vals, 3)
    baseline_predicted_vals[:3] = real_vals[:3]

    plt.plot(dates, real_vals, label='Actual Exchange Rate', color='blue')
    plt.plot(dates, lstm_predicted_vals, label='LSTM Predicted Exchange Rate', color='red')
    plt.plot(dates, baseline_predicted_vals, label='Baseline Predicted Exchange Rate', color='green')

    random_indices = random.sample(range(len(real_vals)), num_dates)
    for idx in random_indices:
        date = data.iloc[train_size + idx]['Date']
        plt.scatter(mdates.date2num(date), real_vals[idx], color='blue')
        plt.scatter(mdates.date2num(date), lstm_predicted_vals[idx], color='red')
        plt.scatter(mdates.date2num(date), baseline_predicted_vals[idx], color='green')
    print(
        f"Date: {date.strftime('%Y-%m-%d')}, Actual: {real_vals[idx]}, LSTM Predicted: {lstm_predicted_vals[idx]}, Baseline Predicted: {baseline_predicted_vals[idx]}")

    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('Exchange Rate')
    plt.legend()
    plt.gcf().autofmt_xdate()
    plt.show()


    lstm_mae = np.mean(np.abs(real_vals - lstm_predicted_vals))
    baseline_mae = np.mean(np.abs(real_vals - baseline_predicted_vals))

    print(f"Average MAE between LSTM predictions and actual values: {lstm_mae}")
    print(f"Average MAE between baseline predictions and actual values: {baseline_mae}")

    return lstm_mae, baseline_mae

In [ ]:
time_steps_list = [3]
performance_metrics = {}

for time_steps in time_steps_list:
    X, y = create_dataset(features_scaled, target_scaled.ravel(), time_steps=time_steps)

    train_size = int(len(X) * 0.96)
    X_train, X_val = X[:train_size], X[train_size:]
    y_train, y_val = y[:train_size], y[train_size:]

    input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))

    x = Bidirectional(LSTM(200, return_sequences=True, kernel_regularizer=l2(0.01)))(input_layer)
    x = Dropout(0.3)(x)
    x = BatchNormalization()(x)

    x = Bidirectional(LSTM(200, return_sequences=True, kernel_regularizer=l2(0.01)))(x)
    x = Dropout(0.3)(x)
    x = BatchNormalization()(x)

    x = Bidirectional(LSTM(200, return_sequences=True, kernel_regularizer=l2(0.01)))(x)
    x = Dropout(0.3)(x)
    x = BatchNormalization()(x)

    query = x
    value = x
    attention = Attention(use_scale=True)([query, value])
    attention = Dropout(0.05)(attention)
    attention = BatchNormalization()(attention)

    x = Bidirectional(LSTM(200, kernel_regularizer=l2(0.01)))(attention)
    x = Dropout(0.3)(x)

    output = Dense(1)(x)

    model = Model(inputs=input_layer, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    history = model.fit(X_train, y_train, epochs=100, batch_size=256,
                        validation_data=(X_val, y_val), verbose=1,
                        callbacks=[early_stop]
                        )

    predicted_val = model.predict(X_val)
    predicted_val_rescaled = scaler_target.inverse_transform(predicted_val)
    real_val_rescaled = scaler_target.inverse_transform(y_val.reshape(-1, 1))

    mse_val = mean_squared_error(real_val_rescaled, predicted_val_rescaled)
    rmse_val = np.sqrt(mse_val)
    mae_val = mean_absolute_error(real_val_rescaled, predicted_val_rescaled)
    r2_val = r2_score(real_val_rescaled, predicted_val_rescaled)

    performance_metrics[time_steps] = {
        'MSE': mse_val,
        'RMSE': rmse_val,
        'MAE': mae_val,
        'R^2': r2_val
    }



    plot_predictions(real_val_rescaled, predicted_val_rescaled,
                     title=f"LSTM Model Predictions for time_steps = {time_steps}")


In [ ]:
for time_steps, metrics in performance_metrics.items():
    print(f"Results for time_steps = {time_steps}:")
    print(f"MSE: {metrics['MSE']}, RMSE: {metrics['RMSE']}, MAE: {metrics['MAE']}, R^2: {metrics['R^2']}\n")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LSTM, BatchNormalization, Attention
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import random
import logging

In [ ]:
def initialize_particles(num_particles, lower_bound, upper_bound):
    particles = np.random.uniform(low=-0.008, high=0.008, size=num_particles)
    weights = np.ones(num_particles) / num_particles
    return particles, weights

In [ ]:
def predict_particles(particles, lstm_predict_function, features):
    predictions = lstm_predict_function(features)
    predicted_particles = predictions.flatten() + np.random.normal(0, 0.001, size=particles.shape)
    return predicted_particles

In [ ]:
def update_particle_weights(particles, weights, actual_value, beta=0.1):
    particle_differences = np.abs(particles - actual_value)
    logging.debug(f"Before update: max weight={np.max(weights)}, min weight={np.min(weights)}")
    weights *= np.exp(-beta * particle_differences)
    weights += 1.e-30000
    weights /= np.sum(weights)
    logging.debug(f"After update: max weight={np.max(weights)}, min weight={np.min(weights)}")
    return weights

In [ ]:
def stratified_resample(particles, weights):
    N = len(particles)
    positions = (np.random.rand(N) + range(N)) / N
    indexes = np.zeros(N, 'i')
    cumulative_sum = np.cumsum(weights)
    i, j = 0, 0
    while i < N:
        if positions[i] < cumulative_sum[j]:
            indexes[i] = j
            i += 1
        else:
            j += 1
    return particles[indexes], np.ones_like(weights) / N

In [ ]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X[i:(i + time_steps), :]
        Xs.append(v)
        ys.append(y[i + time_steps - 1])
    return np.array(Xs), np.array(ys)

In [ ]:
def train_lstm_model(X_train, y_train, X_val, y_val, time_steps):
    input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))
    x = LSTM(200, return_sequences=True, kernel_regularizer=l2(0.01))(input_layer)
    x = Dropout(0.05)(x)
    x = BatchNormalization()(x)
    attention = Attention(use_scale=True)([x, x])
    attention = Dropout(0.05)(attention)
    attention = BatchNormalization()(attention)
    x = LSTM(200, kernel_regularizer=l2(0.01))(attention)
    x = Dropout(0.05)(x)
    output = Dense(1)(x)
    model = Model(inputs=input_layer, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val),
                        callbacks=[early_stop])
    return model

In [ ]:
def lstm_predict_function(model, scaler, features):
    features = np.expand_dims(features, axis=0)
    prediction = model.predict(features)
    return scaler.inverse_transform(prediction).flatten()

In [ ]:
def evaluate_model(model, X_test, y_test, scaler):
    predictions = model.predict(X_test)
    predicted_vals = scaler.inverse_transform(predictions)

    actual_vals = scaler.inverse_transform(y_test.reshape(-1, 1))

    plt.figure(figsize=(10, 5))
    plt.plot(actual_vals, label='Actual Values')
    plt.plot(predicted_vals, label='Predicted Values', alpha=0.7)
    plt.title('LSTM Model Evaluation')
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.legend()
    plt.show()

    mse = mean_squared_error(actual_vals, predicted_vals)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(actual_vals, predicted_vals)
    r2_val = r2_score(actual_vals, predicted_vals)
    print(f"MSE: {mse}, RMSE: {rmse}, MAE: {mae}, R^2:{r2_val}")

In [ ]:
def plot_particle_trajectories(data, real_vals, particle_trajectories, train_size):
    with plt.style.context('ggplot'):
        plt.figure(figsize=(14, 7))
    valid_indices = ~np.isnan(real_vals).flatten()
    dates = data.iloc[train_size:train_size + len(real_vals)]['Date'][valid_indices]
    real_vals = real_vals[valid_indices]

    dates_mapped = dates.map(mdates.date2num)
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

    plt.plot(dates_mapped, real_vals, label='Actual Log Returns', color='blue', alpha=0.75)

    for particles_at_time_t in particle_trajectories.T:
        plt.scatter(dates_mapped, particles_at_time_t, color='red', alpha=0.05)

    plt.title("Particle Filter Predictions over Time")
    plt.xlabel('Time')
    plt.ylabel('Log Returns')
    plt.legend()
    plt.gcf().autofmt_xdate()
    plt.show()

In [ ]:
def main():
    data = pd.read_csv(r'Database2.csv')
    data['Date'] = pd.to_datetime(data['Date'])
    data.sort_values('Date', inplace=True)
    data['Log_Returns'] = np.log(data['ExchangeRate'] / data['ExchangeRate'].shift(1))
    data['Price_Log_Return'] = np.log(data['Price'] / data['Price'].shift(1))
    data['STI_Log_Return'] = np.log(data['STI'] / data['STI'].shift(1))
    data['GoldPrice_Log_Return'] = np.log(data['GoldPrice'] / data['GoldPrice'].shift(1))
    data['USD_EUR_ExchangeRate_Log_Return'] = np.log(
    data['USD_EUR_ExchangeRate'] / data['USD_EUR_ExchangeRate'].shift(1))
    data['USD_CNY_ExchangeRate_Log_Return'] = np.log(
    data['USD_CNY_ExchangeRate'] / data['USD_CNY_ExchangeRate'].shift(1))
    data['impact_score_Log_Return'] = np.log(data['impact_score'] / data['impact_score'].shift(1))
    data['ExchangeRate_Short_MA'] = data['ExchangeRate'].rolling(window=20, min_periods=1).mean()
    data['ExchangeRate_Long_MA'] = data['ExchangeRate'].rolling(window=80, min_periods=1).mean()
    data['ExchangeRate_Long_MA_Log_Return'] = np.log(
    data['ExchangeRate_Long_MA'] / data['ExchangeRate_Long_MA'].shift(1))
    data['ExchangeRate_Short_MA_Log_Return'] = np.log(
    data['ExchangeRate_Short_MA'] / data['ExchangeRate_Short_MA'].shift(1))
    selected_features = [
        'Price_Log_Return', 'STI_Log_Return', 'GoldPrice_Log_Return',
        'USD_EUR_ExchangeRate_Log_Return', 'USD_CNY_ExchangeRate_Log_Return',
        'ExchangeRate_Long_MA_Log_Return', 'ExchangeRate_Short_MA_Log_Return',
        'impact_score_Log_Return'
        ]
    features = data[selected_features]
    target = data['Log_Returns']
    features = data[selected_features].dropna()
    target = data['Log_Returns'].dropna()
    scaler_features = MinMaxScaler()
    features_scaled = scaler_features.fit_transform(features)
    scaler_target = MinMaxScaler()
    target_scaled = scaler_target.fit_transform(target.values.reshape(-1, 1))
    X, y = create_dataset(features_scaled, target_scaled.ravel(), time_steps=3)
    train_size = int(len(X) * 0.96)
    X_train, X_val = X[:train_size], X[train_size:]
    y_train, y_val = y[:train_size], y[train_size:]
    model = train_lstm_model(X_train, y_train, X_val, y_val, 3)
    predicted_val = model.predict(X_val)
    predicted_val_rescaled = scaler_target.inverse_transform(predicted_val)
    real_val_rescaled = scaler_target.inverse_transform(y_val.reshape(-1, 1))

    num_particles = 1000
    particles, weights = initialize_particles(num_particles, lower_bound=-0.05, upper_bound=0.05)
    particle_trajectories = np.zeros((len(X_val), num_particles))

    for i in range(len(X_val)):
        current_features = X_val[i]
        actual_value = y_val[i]

        particles = predict_particles(particles, lambda features: lstm_predict_function(model, scaler_target, features),
                                      current_features)
        weights = update_particle_weights(particles, weights, actual_value)
        particles, weights = stratified_resample(particles, weights)
        particle_trajectories[i] = particles

    plot_particle_trajectories(data, real_val_rescaled, particle_trajectories, train_size)
    evaluate_model(model, X_val, y_val, scaler_target)

In [ ]:
if __name__ == "__main__":
    main()